Author: Just-Q-it

In [5]:
import pytket as tk
from pytket import passes as tkp
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter as print_circ
import numpy as np
from pytket import OpType

In [6]:
#function to decompose circuit into Us
def decomposition(circ: Circuit):
        n_qubits = circ.n_qubits #original # of qubits
        h = circ.n_gates_of_type(OpType.H) # of H gates

        new_circ = Circuit(n_qubits) #any circuit
        u_list = [] # to store all Us
        counter = 0 # to slice after each H gate

        for i in range(h):
                for cmd in circ.get_commands()[counter:]:
                        qubit_list = cmd.qubits 
                        if cmd.op.type != OpType.H:
                                new_circ.add_gate(cmd.op.type, cmd.op.params, qubit_list)
                                counter += 1
                        else:
                                counter += 1
                                break
                
                
                u_list.append(new_circ)
                new_circ = Circuit(n_qubits)
        
        return u_list

In [76]:
# Doing gadgetisation
def h_gadgetisation2(circ: Circuit):

    n_qubits = circ.n_qubits # original # of qubits
    h = circ.n_gates_of_type(OpType.H) # # of H gates
    u_list = decomposition(circ) # getting all U_i
    new_circ = Circuit(n_qubits) # Define a replacement circuit
    ancilla = new_circ.add_q_register("a", h)
    cr = new_circ.add_c_register("c",h)
    

    
    counter = 0

    for i in range(h):
        new_circ.H(ancilla[i])

    for cmd in circ.get_commands():
        #qubit_list = cmd.qubits # accesing the qubit of cmd
        if cmd.op.type == OpType.H:
            new_circ.append(u_list[counter])
            #print(cmd.qubits)
            q_index = cmd.qubits[0]
            new_circ.S(q_index).S(ancilla[counter]).CX(q_index, ancilla[counter]).Sdg(ancilla[counter]).CX(ancilla[counter],q_index).CX(q_index,ancilla[counter])
            
            #Measuring
            new_circ.Measure(ancilla[counter], cr[counter])

            #Condition
            #u_dagger = u_list[counter].dagger()
            #udagger_box = u_list.appendCircbox(u_list.append())
            
            new_circ.X(q_index, condition_bits=[cr[counter]], condition_value=1)
            #new_circ.add_gate(q_index, condition_bits=[cr[counter]], condition_value=1)
            #new_circ = new_circ.append(u_list[counter])
            
            counter += 1

    return new_circ

In [77]:
test1 = tk.Circuit(5).H(1).CX(2,3).CX(2,1).CY(1,2).H(2).Z(3).CX(2,1).H(1).CX(2,3).CX(2,1).CZ(1,2).H(2).Z(3).CX(2,1)
#test1.get_commands()
print_circ(test1)

In [78]:
DecompseZPass = tkp.CustomPass(h_gadgetisation2) # Define our pass
DecompseZPass.apply(test1) # Apply our pass to the test Circuit
#test1.get_commands()
print_circ(test1)

In [10]:
test1.qubits[0]

q[0]